In [1]:
#preprocess
import json
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# deploy

In [16]:
import boto3
import csv
import json
import sagemaker
import os
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader,s3_path_join



sess = sagemaker.Session()
role = get_execution_role()

prefix='stary-datalab-modela'

bucket = sess.default_bucket() 


dataset_csv_file='../model_a_data/test.csv'
dataset_jsonl_file="stary_data.jsonl"

with open(dataset_csv_file, "r+") as infile, open(dataset_jsonl_file, "w+") as outfile:
    reader = csv.DictReader(infile)
    for row in reader:
        row1={"inputs": ''}
        row1["inputs"] = row["sentence1_key"]
        json.dump(row1, outfile)
        outfile.write('\n')

input_s3_path = s3_path_join("s3://",bucket,"stary-datalab-modela/test")
s3_file_uri = S3Uploader.upload(dataset_jsonl_file,input_s3_path)

print(f"{dataset_jsonl_file} uploaded to {s3_file_uri}")


stary_data.jsonl uploaded to s3://sagemaker-us-east-1-734836471744/stary-datalab-modela/test/stary_data.jsonl


In [25]:
from sagemaker.huggingface.model import HuggingFaceModel


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-734836471744/stary-modela-bert-base-epoch5-2022-07-19-01-37-27-714/output/model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version='py36', # python version used
)

output_s3_path = s3_path_join("s3://",bucket,"stary-datalab-modela/output")
# create Transformer to run our batch job
batch_job = huggingface_model.transformer(
    instance_count=1,
    instance_type='ml.m5.4xlarge',
    output_path=output_s3_path, # we are using the same s3 path to save the output with the input
    strategy='SingleRecord')


In [27]:
# starts batch transform job and uses s3 data as input
batch_job.transform(
    data=s3_file_uri,
    content_type='application/json',    
    split_type='Line')

..................................2022-07-19T03:14:14.070:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=SINGLE_RECORD
2022-07-19 03:14:13,493 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
2022-07-19 03:14:13,493 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /opt/conda/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 16
Max heap size: 13444 M
Python executable: /opt/conda/bin/python3.6
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 16
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Preload model: false
Prefer direct buffer: false
2022-07-19 03:14:13,546 [WARN ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerLi

In [29]:
import json
from sagemaker.s3 import S3Downloader
from ast import literal_eval
# creating s3 uri for result file -> input file + .out
output_file = f"{dataset_jsonl_file}.out"
output_path = s3_path_join(output_s3_path,output_file)

# download file
S3Downloader.download(output_path,'.')

batch_transform_result = []
with open(output_file) as f:
    for line in f:
        # converts jsonline array to normal array
        line = "[" + line.replace("[","").replace("]",",") + "]"
        batch_transform_result = literal_eval(line) 
        
# print results 
print(batch_transform_result[:10])

[{'label': 0, 'score': 0.9998965263366699}, {'label': 0, 'score': 0.999894380569458}, {'label': 1, 'score': 0.9940595030784607}, {'label': 0, 'score': 0.9998974204063416}, {'label': 0, 'score': 0.9998978972434998}, {'label': 0, 'score': 0.9998975992202759}, {'label': 0, 'score': 0.9998980760574341}, {'label': 0, 'score': 0.9998981952667236}, {'label': 0, 'score': 0.9846323132514954}, {'label': 1, 'score': 0.9924623370170593}]


# Real-time inference

In [5]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sagemaker-us-east-1-734836471744/stary-modela-bert-base-epoch5-2022-07-19-01-37-27-714/output/model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [6]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)

-------!

In [17]:
from sagemaker.predictor import Predictor
import sagemaker

predictor = Predictor(endpoint_name="stray-distilbert-base-uncased-imdb")
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [29]:
%%time
# example request, you always need to define "inputs"
import time


data = {
   "inputs": "I\'m sorry. I got carried away,",
}

# request
predictor.predict(data)


CPU times: user 4.55 ms, sys: 21 µs, total: 4.57 ms
Wall time: 21.6 ms


[{'label': 0, 'score': 0.9998857975006104}]

In [19]:
import pandas as pd
df = pd.read_csv('../model_a_data/test.csv')

In [20]:
res = []
for i in df['sentence1_key']:
    data = {"inputs": i}

    # request
    res.append(predictor.predict(data))

In [21]:
#p f r
from sklearn.metrics import classification_report

y_pred = [int(i[0]['label']) for i in res]
y_true = df['label']
 
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      1795
           1       0.89      0.94      0.91       651

    accuracy                           0.95      2446
   macro avg       0.93      0.95      0.94      2446
weighted avg       0.95      0.95      0.95      2446



In [22]:
df.tail(10)

,label,sentence1_key
2436,0,he said.
2437,0,"She unfolded it, finding a number on it with a..."
2438,0,"""They drugged us with something… when we woke ..."
2439,0,"""My name is Edward Corey. Can you please tell ..."
2440,1,"""I’m taking you out on dinner first… It’s been..."
2441,1,"""We’re not gonna do anything,"" ; \n"
2442,1,"""You want to play games, Sassenach? Bring it on."""
2443,1,"""What the hell does he want?"""
2444,0,"""Ezra was adopted by the pack Alpha, Alistair ..."
2445,0,"""I'm sorry that I didn't give it back after yo..."


In [24]:
y_pred[:10]

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]

In [26]:
df['sentence1_key'][2]

'"I\'m sorry. I got carried away," '

In [24]:
df['sentence1_key'][370]

'"What do you mean by that?" She asks me surprised. '